In [1]:
import numpy as  np
import pandas as pd
from random import seed, random
import tensorflow as tf
from keras.models import Sequential

C:\Users\Pink\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data=[[[i+1+j] for i in range(3)] for j in range(4)]
data=np.array(data)

In [3]:
target=[(i+4) for i in range(4)]
target=np.array(target)

In [4]:
data,target

(array([[[1],
         [2],
         [3]],
 
        [[2],
         [3],
         [4]],
 
        [[3],
         [4],
         [5]],
 
        [[4],
         [5],
         [6]]]), array([4, 5, 6, 7]))

In [5]:
print('Shape of data -->',data.shape,'\nShape of target-->',target.shape)

Shape of data --> (4, 3, 1) 
Shape of target--> (4,)


In [6]:
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

In [7]:
def sigmoidPrime(output):
    return output*(1-output)

#### Forward-Propogation in RNN
#### How it's Works

In [8]:
test=data[0]
hidden_dim=10
#input_dim = data.shape[1]
input_dim = data.shape[1]
output_dim = 1

print('Shape of hidden_dim is-->',hidden_dim,'\nShape of input_dim is-->',data.shape[1],'\nShape of input_dim is-->',1)
wxh = 2*np.random.random((hidden_dim,input_dim)) - 1
whh = 2*np.random.random((hidden_dim,hidden_dim)) - 1
why = 2*np.random.random((hidden_dim,output_dim)) - 1

Shape of hidden_dim is--> 10 
Shape of input_dim is--> 3 
Shape of input_dim is--> 1


In [9]:
### it is a previous state you may called it 0-State
hprev=np.zeros(hidden_dim)
hprev

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
## When data=data[0]
"""
 Input=1,2,3
 Target=4
"""

xs_0=sigmoid(np.dot(wxh,data[0])+np.dot(whh,hprev).reshape(hidden_dim,1))
os_0=sigmoid(np.dot(why.T,xs_0))
print('os_0 equals to',os_0)

## When data=data[1]

"""
 Input=2,3,4
 Target=5

"""

xs_1=sigmoid(np.dot(wxh,data[1])+np.dot(whh,xs_0).reshape(hidden_dim,1))
os_1=sigmoid(np.dot(why.T,xs_1))
print('os_1 equals to',os_1)


## When data=data[2]

"""
 Input=3,4,5
 Target=6

"""

xs_2=sigmoid(np.dot(wxh,data[2])+np.dot(whh,xs_1).reshape(hidden_dim,1))
os_2=sigmoid(np.dot(why.T,xs_2))
print('os_2 equals to',os_2)

## When data=data[3]

"""
 Input=4,5,6
 Target=7

"""

xs_3=sigmoid(np.dot(wxh,data[3])+np.dot(whh,xs_2).reshape(hidden_dim,1))
os_3=sigmoid(np.dot(why.T,xs_3))
print('os_3 equals to',os_3)

T=len(data)
xs = np.zeros((T,hidden_dim))
os= np.zeros(len(data))
xs = np.zeros((T,hidden_dim))
xs[-1]=np.zeros((hidden_dim))
os = np.zeros((len(data),1))
os= np.zeros(len(data))
print('-------------------Above and Below sequences are same this is how RNN Works-----------------')
for i in range(len(data)):
    xs[i]=sigmoid(np.dot(wxh,data[i])+(np.dot(whh,xs[i-1])).reshape(hidden_dim,1)).reshape(hidden_dim)
    os[i]=sigmoid(np.dot(why.T,xs[i]))
print(os)

os_0 equals to [[0.44286146]]
os_1 equals to [[0.47999096]]
os_2 equals to [[0.4510859]]
os_3 equals to [[0.45500739]]
-------------------Above and Below sequences are same this is how RNN Works-----------------
[0.44286146 0.47999096 0.4510859  0.45500739]


#### RNN Code is Started

In [11]:
def RNNSeq(lr,epoch):
    
    
    hidden_dim=10
    input_dim = data.shape[1]
    output_dim = 1

    """
    Total Number of T is equal to length of data and we are adding one because we need to consider initial state
    which is all zero.
    int_s is a initial state also know as (h0)

    """ 

    np.random.seed(1)
    wxh = 2*np.random.random((hidden_dim,input_dim)) - 1
    whh = 2*np.random.random((hidden_dim,hidden_dim)) - 1
    why = 2*np.random.random((hidden_dim,output_dim)) - 1

    T = len(data)

    xs = np.zeros((T,hidden_dim))
    xs[-1]=np.zeros((hidden_dim))
    os = np.zeros((len(data),1))
    os= np.zeros(len(data))

    ### Starting of Gradient descent
    for ep in range(epoch):
        
        ### Run the code equals to length of data

        for i in range(len(data)):
            
            """ 
            ## Reshape is just broadcasting of Data
            Basically xs[i]=sigmoid(np.dot(wxh,data[i])+(np.dot(whh,xs[i-1])))

            """ 
            xs[i]=sigmoid(np.dot(wxh,data[i])+(np.dot(whh,xs[i-1])).reshape(hidden_dim,1)).reshape(hidden_dim)

            os[i]=sigmoid(np.dot(why.T,xs[i]))

            cost=np.sum((os[i]-target)**2)/2

            """
            Back-propogation through Time starts here

            Using Gradient Descent

            backward pass: compute gradients going backwards    
            initalize vectors for gradient values for each set of weights 

            """

            djdwhy=((os[i]-target[i])*sigmoidPrime(os[i])*xs[i]).reshape(hidden_dim,1)

            djdwxh=((os[i]-target[i])*sigmoidPrime(os[i])*why)*sigmoidPrime(xs[i]).reshape(hidden_dim,1)*data[i].T

            djdwhh=(((os[i]-target[i])*sigmoidPrime(os[i])*why)*sigmoidPrime(xs[i]).reshape(hidden_dim,1))*(xs[i-1].reshape(hidden_dim,1))


            ## SGD of why
            why=why-lr*djdwhy

            ## SGD of wxh
            wxh=wxh-lr*djdwxh

            ## SGD of whh
            whh=whh-lr*djdwhh
            
            
            # Assign new parameters to the model
            
            model={}
            
            model = {'Iteration':ep,'RMSE Loss':cost,'why': why,'whh': whh,'whx':wxh}
            
        if ep%400 ==0:
                
            print('Iteration =',ep,'\n','RMSE Loss',cost,'\n','why','\n',why,'\n','whh','\n',whh,'\n','wxh','\n',wxh)
            print('------------------------------------------------------------------------------------------')
                
    return model


In [12]:
RNNSeq(lr=.001,epoch=1200)

Iteration = 0 
 RMSE Loss 56.53440780299797 
 why 
 [[ 0.17171067]
 [ 0.93920592]
 [ 0.12219894]
 [-0.95876846]
 [ 0.60157885]
 [-0.53008676]
 [ 0.61507092]
 [-0.21989884]
 [ 0.73099304]
 [ 0.49515369]] 
 whh 
 [[-0.80330488 -0.1577833   0.91578051  0.06633202  0.38375568 -0.36896729
   0.37300331  0.66925279 -0.963422    0.50029008]
 [ 0.9777222   0.49633133 -0.43911199  0.57855868 -0.79354796 -0.10421292
   0.81719103 -0.41277168 -0.4244493  -0.73994283]
 [-0.96126583  0.35767132 -0.57674351 -0.46890642 -0.01685342 -0.89327465
   0.14823547 -0.70654259  0.17861133  0.39951698]
 [-0.79561453 -0.17217141  0.38851693 -0.17192485 -0.90037647  0.07150942
   0.3273059   0.02949483  0.88890612  0.17282669]
 [ 0.80681625 -0.72503817 -0.72143489  0.614795   -0.20463391 -0.66927919
   0.85502958 -0.30445586  0.50163662  0.45200839]
 [ 0.76650356  0.24723579  0.50177624 -0.30231194 -0.46025284  0.79166381
  -0.14392625  0.92957147  0.32677437  0.24328281]
 [-0.77042754  0.89905903 -0.10009522  

{'Iteration': 1199,
 'RMSE Loss': 43.11317567574103,
 'why': array([[ 0.20972528],
        [ 0.94048867],
        [ 0.13396726],
        [-0.57751604],
        [ 0.80744916],
        [ 0.28621659],
        [ 1.0473676 ],
        [ 0.66986619],
        [ 1.59083792],
        [ 1.0015603 ]]),
 'whh': array([[-0.80294759, -0.15742601,  0.9161378 ,  0.06668931,  0.38411297,
         -0.36861   ,  0.37336059,  0.66961008, -0.96306471,  0.50064737],
        [ 0.97772347,  0.4963326 , -0.43911072,  0.57855995, -0.79354669,
         -0.10421165,  0.8171923 , -0.41277041, -0.42444803, -0.73994156],
        [-0.96124649,  0.35769066, -0.57672417, -0.46888709, -0.01683409,
         -0.89325531,  0.14825481, -0.70652325,  0.17863067,  0.39953632],
        [-0.84624861, -0.22280549,  0.33788285, -0.22255893, -0.95101055,
          0.02087535,  0.27667182, -0.02113924,  0.83827205,  0.12219261],
        [ 0.8355989 , -0.69625553, -0.69265224,  0.64357764, -0.17585126,
         -0.64049654,  0.883812